<a href="https://colab.research.google.com/github/welcomeglory/Python/blob/master/WineANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install tqdm
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

2.17.0


In [5]:
# red? white?
class WineANN(tf.keras.Model):
    def __init__(self):
        super(WineANN, self).__init__()
        # 층을 몇개, 노드 몇개 : 2층
        # 입력층 12개 x : input층 Dense : layer층 weight - 176개 생성
        self.hidden1 = tf.keras.layers.Dense(8,
                                             activation='relu',
                                             kernel_initializer = self.weight_initialization(),
                                             bias_initializer = self.weight_initialization())
        self.hidden2 = tf.keras.layers.Dense(8,
                                             activation='relu',
                                             kernel_initializer = self.weight_initialization(),
                                             bias_initializer = self.weight_initialization())
        self.result = tf.keras.layers.Dense(2,
                                            activation=None,
                                            kernel_initializer = self.weight_initialization(),
                                            bias_initializer = self.weight_initialization())

    def weight_initialization(self):
        return tf.keras.initializers.RandomNormal(mean=0, stddev=1)

    def call(self, x, training=False):
        # 층에 순서대로 계산
        y1 = self.hidden1(x)
        y2 = self.hidden2(y1)
        logits = self.result(y2) #
        sy = tf.nn.softmax(logits) # 확률이 높은것 고름
        return logits, sy

In [2]:
@tf.function
def cross_entropy_loss(logits, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)) # 에러률 ex)0.2 0.1 ....

learningRate = 0.001 # learningRate가 작을수록 섬세하다
optimizer = tf.optimizers.Adam(learningRate) # 미분법(경사하강법), Adagrad, Momentum,....

@tf.function
def backward(model, x, y):
    with tf.GradientTape() as grad:
        logits, sy = model(x, training=True) #call함수 부르는 곳
        loss = cross_entropy_loss(logits, y)
    grads = grad.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

@tf.function
def accuracy(predY, y):
    correct_pred = tf.equal(tf.argmax(predY, axis=1), tf.argmax(y, axis=1))
    acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return acc

In [3]:
# Training function
def train_learning(epochs, frequency, trainDataIter, model, XTest, yTest): # frequency 에러출력 빈도
    y_loss = []
    for epoch in tqdm(range(epochs)):
        bx, by = next(trainDataIter)
        loss = backward(model, bx, by)
        y_loss.append(loss)
        if epoch % frequency == 0:
            predY, sy = model(bx, training=False)
            acc = accuracy(predY, by)
            print(f"Epoch: {epoch+1} ===> Loss: {loss:.4f}, Accuracy: {acc:.4f}")
    predY, sy = model(XTest, training=False)
    acc = accuracy(predY, yTest)
    print(f"Test Accuracy: {acc:.4f}")
    return y_loss

In [6]:
red_path = "/content/drive/MyDrive/세종교육/winequality-red.csv"
white_path = "/content/drive/MyDrive/세종교육/winequality-white.csv"
red_df = pd.read_csv(red_path, sep=';')
red_df['label'] = 0
white_df = pd.read_csv(white_path, sep=';')
white_df['label'] = 1
wine_df = pd.concat([red_df, white_df])
wine_df.reset_index(drop=True, inplace=True)
matrix = wine_df.values
X = matrix[:, :-1]
X = (X-X.min(axis=0)) / (X.max(axis=0)-X.min(axis=0))
# print(X)
y = matrix[:, -1]


XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=42)
XTrain, XTest = XTrain.astype('float32'), XTest.astype('float32')

# One-hot encode y
yTrain = tf.one_hot(yTrain.flatten(), 2)
yTest = tf.one_hot(yTest.flatten(), 2)

train_data = tf.data.Dataset.from_tensor_slices((XTrain, yTrain))
train_data = train_data.shuffle(buffer_size=XTrain.shape[0]).batch(300).repeat()
trainDataIter = iter(train_data)
bx, by = next(trainDataIter)

epochs = 5000
frequency = 150

model = WineANN()
y_loss = train_learning(epochs, frequency, trainDataIter, model, XTest, yTest)


  0%|          | 1/5000 [00:01<1:36:27,  1.16s/it]

Epoch: 1 ===> Loss: 1.2363, Accuracy: 0.2867


  4%|▍         | 212/5000 [00:01<00:20, 238.37it/s]

Epoch: 151 ===> Loss: 0.5490, Accuracy: 0.8033


  7%|▋         | 362/5000 [00:02<00:12, 370.60it/s]

...

Epoch: 4651 ===> Loss: 0.0119, Accuracy: 1.0000


 97%|█████████▋| 4842/5000 [00:11<00:00, 311.39it/s]

Epoch: 4801 ===> Loss: 0.0571, Accuracy: 0.9833


100%|██████████| 5000/5000 [00:12<00:00, 408.85it/s]

Epoch: 4951 ===> Loss: 0.0068, Accuracy: 1.0000
Test Accuracy: 0.9897
